In [7]:
# only for first time running
# !pip install ortools

# Automated Nurse Scheduling with ortools
### First, import the necessary libraries and data

In [2]:
import json
from ortools.sat.python import cp_model

In [3]:
file_name = "Schedule_sample.json"
data_file = open(file_name,'r')
data = json.load(data_file)
print(len(data))

#check len of the data, should be 141 in the sample

141


### Convert data into a list
Also remember to check for duplicates and exclude accordingly

In [4]:
staff_ids = []
skill_ids = []
ss_ids = []
duty_ids = []
shift_duties_ids = []

staff_list = []
skills_list = []
staff_skills_list = []
duties_list = []
shift_duties_list = []

for item in data:
    #full data is a list, item is a dict
    if 'staff' in item:
        if item["staff"]["staff_id"] not in staff_ids:
            staff_list.append(item["staff"])
            staff_ids.append(item["staff"]["staff_id"])
        else:
            print('found duplicate id for staff id', item["staff"]["staff_id"], "where staff_ids =",staff_ids)

    elif 'skills' in  item:
        if item["skills"]["skill_id"] not in skill_ids:
            skills_list.append(item["skills"])
            skill_ids.append(item["skills"]["skill_id"])
        else:
            print('found duplicate id for skill id', item["skills"]["skill_id"], "where skill_ids =",skill_ids)
        
    elif 'staff_skills' in item:
        if item["staff_skills"]["ss_id"] not in ss_ids:
            staff_skills_list.append(item["staff_skills"])
            ss_ids.append(item["staff_skills"]["ss_id"])
        else:
            print('found duplicate id for staff skills')

    elif 'duties' in  item:
        if item["duties"]["duty_id"] not in duty_ids:
            duties_list.append(item["duties"])
            duty_ids.append(item["duties"]["duty_id"])
        else:
            print('found duplicate id for duty')

    elif 'shift_duties' in  item:
        if (item["shift_duties"]["duty_id"],item["shift_duties"]["duty_name"]) not in shift_duties_ids:
            shift_duties_list.append(item["shift_duties"])
            shift_duties_ids.append((item["shift_duties"]["duty_id"],item["shift_duties"]["duty_name"]))
        else:
            print('found duplicate id for shift duties')


            
num_staffs = len(staff_ids)
num_skills = len(skill_ids)
num_staff_skills = len(ss_ids)
num_duties = len(duty_ids)
num_shift_duties = len(shift_duties_ids)


found duplicate id for staff id 21 where staff_ids = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]


### Parse staff_qualifications into a boolean hash table & check quantity for each data parameter

In [6]:
print("staffs =",num_staffs,"\nskills =", num_skills,"\nstaff skills=",\
      num_staff_skills, "\nduties =",num_duties, "\nshift duties =",num_shift_duties)

staffs = 25 
skills = 3 
staff skills= 54 
duties = 29 
shift duties = 29


In [7]:
staff_qualifications = [[] for x in range(num_staffs)]

for item in staff_skills_list:
    staff_id = item['staff_id']
    skill_id = item['skill_id']
    staff_qualifications[staff_id-1].append(skill_id)

print("staffs =",num_staffs,"\nskills =", num_skills,"\nstaff skills=",\
      num_staff_skills, "\nduties =",num_duties, "\nshift duties =",num_shift_duties)

print(staff_qualifications)

[[1, 2], [1, 2], [1, 2], [1, 2, 3], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2, 3], [1, 2], [1, 2, 3], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2], [1, 2, 3], [1, 2], [1, 2], [2, 3], [1, 2]]


In [8]:
staff_qualifications_dict = {}
for i in range(len(staff_qualifications)):
    qualifications = staff_qualifications[i]
    staff_qualifications_dict[i] = [1 in qualifications, 2 in qualifications, 3 in qualifications]
    
print(staff_qualifications_dict)
#for staff id 0: qualified for CT/MRI/DSA = T/T/F

{0: [True, True, False], 1: [True, True, False], 2: [True, True, False], 3: [True, True, True], 4: [True, True, False], 5: [True, True, False], 6: [True, True, False], 7: [True, True, False], 8: [True, True, False], 9: [True, True, True], 10: [True, True, False], 11: [True, True, True], 12: [True, True, False], 13: [True, True, False], 14: [True, True, False], 15: [True, True, False], 16: [True, True, False], 17: [True, True, False], 18: [True, True, False], 19: [True, True, False], 20: [True, True, True], 21: [True, True, False], 22: [True, True, False], 23: [False, True, True], 24: [True, True, False]}


In [13]:
staff_qualifications_TF = []
for staff_id in range(num_staffs):
    qualified = []
    for duty_qual in staff_qualifications_dict[staff_id]:
        if duty_qual:
            qualified.append(1)
        else:
            qualified.append(0)
            
    staff_qualifications_TF.append(qualified)
    
print(staff_qualifications_TF)
#another way of representing staff_qualifications_dict

[[1, 1, 0], [1, 1, 0], [1, 1, 0], [1, 1, 1], [1, 1, 0], [1, 1, 0], [1, 1, 0], [1, 1, 0], [1, 1, 0], [1, 1, 1], [1, 1, 0], [1, 1, 1], [1, 1, 0], [1, 1, 0], [1, 1, 0], [1, 1, 0], [1, 1, 0], [1, 1, 0], [1, 1, 0], [1, 1, 0], [1, 1, 1], [1, 1, 0], [1, 1, 0], [0, 1, 1], [1, 1, 0]]


### Double check to confirm that for each duty, the required staff per day remains the same in week 1 and week 2

In [13]:
#check if each week is the same in shift duties  
# shift_duties_list

for item in shift_duties_list:
    week1_shifts = item["shift_number"][:7]
    week2_shifts = item["shift_number"][7:]
    # print(week1_shifts, week2_shifts)
    
    if week1_shifts == week2_shifts:
        item["shift_number"] = item["shift_number"][:7]
    else:
        # print("not same for duty id:",item["duty_id"])
        # because of properties of list that point to same item in memory, it's ok to uncomment and run the code above one time as it returns no error
        # will run just fine first time, but not afterwards
        pass

print(shift_duties_list)

[{'duty_id': 1, 'duty_name': 'CT', 'shift_number': [3, 3, 3, 3, 3, 0, 0]}, {'duty_id': 2, 'duty_name': 'CT', 'shift_number': [0, 0, 0, 0, 0, 2, 0]}, {'duty_id': 3, 'duty_name': 'CT2', 'shift_number': [1, 1, 1, 1, 1, 0, 0]}, {'duty_id': 4, 'duty_name': 'CT2', 'shift_number': [0, 0, 0, 0, 0, 1, 0]}, {'duty_id': 5, 'duty_name': 'CT3', 'shift_number': [1, 1, 1, 1, 1, 0, 0]}, {'duty_id': 6, 'duty_name': 'CT3', 'shift_number': [0, 0, 0, 0, 0, 1, 0]}, {'duty_id': 7, 'duty_name': 'CT E', 'shift_number': [1, 1, 1, 1, 1, 1, 0]}, {'duty_id': 8, 'duty_name': 'CT E', 'shift_number': [0, 0, 0, 0, 0, 0, 1]}, {'duty_id': 9, 'duty_name': '8-2', 'shift_number': [0, 0, 0, 0, 0, 0, 2]}, {'duty_id': 10, 'duty_name': 'CT N', 'shift_number': [1, 1, 1, 1, 1, 1, 1]}, {'duty_id': 11, 'duty_name': 'MR', 'shift_number': [1, 1, 1, 1, 1, 0, 0]}, {'duty_id': 12, 'duty_name': 'MR', 'shift_number': [0, 0, 0, 0, 0, 2, 0]}, {'duty_id': 13, 'duty_name': 'MR OP', 'shift_number': [0, 0, 0, 0, 0, 0, 0]}, {'duty_id': 14, 'du

### Parsing Duties into a list
#### Duties:
duty_id - id <br>
duty_name - CT/MRI/Angio <br>
duty_type - 1:CT, 2:MRI, 3:Angio <br>
category - 1:Mon-Fri, 2:Sat, 3:Sun <br>
duration - hours <br>
pattern - Reg, Evening, Night

In [14]:
#Sample showcase of a duty
el_index = 0
print(duties_list[el_index])
print("==================================================================")
print(shift_duties_list[el_index])


{'duty_id': 1, 'duty_name': 'CT', 'duty_type': 1, 'category': 1, 'duration': 7.5, 'pattern': 1}
{'duty_id': 1, 'duty_name': 'CT', 'shift_number': [3, 3, 3, 3, 3, 0, 0]}


In [15]:
duty_details = []
for i in range(num_duties):
    duty = duties_list[i]
    duty_type = duty["duty_type"]
    duty_name = duty["duty_name"]
    duration = duty["duration"]
    pattern = duty["pattern"]
    
    if type(duration) != int:
#         print(duration)
        duration += 1
        duration = int(duration)
    
    daily_staff_req = shift_duties_list[i]["shift_number"]
    particulars = [duty_type, duty_name, duration, pattern, daily_staff_req]
    duty_details.append(particulars)
    
print(duty_details[26])
#0: type (CT/MRI/DSA)
#1: name
#2: duration
#3: pattern
#4: num staff per day

[2, 'MRI N', 10, 3, [1, 1, 1, 1, 1, 1, 1]]


In [17]:
duty_details

[[1, 'CT', 8, 1, [3, 3, 3, 3, 3, 0, 0]],
 [1, 'CT', 5, 1, [0, 0, 0, 0, 0, 2, 0]],
 [1, 'CT2', 8, 1, [1, 1, 1, 1, 1, 0, 0]],
 [1, 'CT2', 5, 1, [0, 0, 0, 0, 0, 1, 0]],
 [1, 'CT3', 8, 1, [1, 1, 1, 1, 1, 0, 0]],
 [1, 'CT3', 5, 1, [0, 0, 0, 0, 0, 1, 0]],
 [1, 'CT E', 7, 2, [1, 1, 1, 1, 1, 1, 0]],
 [1, 'CT E', 6, 2, [0, 0, 0, 0, 0, 0, 1]],
 [1, '8-2', 5, 1, [0, 0, 0, 0, 0, 0, 2]],
 [1, 'CT N', 10, 3, [1, 1, 1, 1, 1, 1, 1]],
 [2, 'MR', 8, 1, [1, 1, 1, 1, 1, 0, 0]],
 [2, 'MR', 5, 1, [0, 0, 0, 0, 0, 2, 0]],
 [2, 'MR OP', 8, 1, [0, 0, 0, 0, 0, 0, 0]],
 [2, 'MR OP', 5, 1, [0, 0, 0, 0, 0, 1, 0]],
 [2, 'MR OP2', 8, 1, [1, 1, 1, 1, 1, 0, 0]],
 [2, 'MR3', 8, 1, [1, 1, 1, 1, 1, 0, 0]],
 [2, 'MR3', 5, 1, [0, 0, 0, 0, 0, 1, 0]],
 [2, 'MR1', 8, 1, [1, 1, 1, 1, 1, 0, 0]],
 [2, 'MR1', 5, 1, [0, 0, 0, 0, 0, 1, 0]],
 [2, 'MR Coordinator', 8, 1, [1, 1, 1, 1, 1, 0, 0]],
 [2, 'MR 2', 7, 2, [1, 1, 1, 1, 1, 0, 0]],
 [2, 'MR E', 7, 2, [1, 1, 1, 1, 1, 1, 0]],
 [2, 'MR E', 6, 2, [0, 0, 0, 0, 0, 0, 1]],
 [2, 'MR 1.30

In [16]:
#create duty_type to visualize the distribution of duty_id and make for easier reference later

duty_types = []
for duty in range(num_duties):
    duty_type = duty_details[duty][0]
    duty_types.append(duty_type)
print(duty_types)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3]


### Create a hash map to check if a staff is qualified to take on a specific duty

In [17]:
staff_duties_quals = {}

for staff_id in range(num_staffs):
    qualifications = staff_qualifications_dict[staff_id]
    quals = []
    for duty in range(num_duties):
        is_qualified = False
        duty_type = duty_types[duty]
    
        quals.append(qualifications[duty_type-1])
            
    
    staff_duties_quals[staff_id] = quals
    
print(staff_duties_quals)
#staff 0 qualified for everything except duty 27 and 28

{0: [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False], 1: [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False], 2: [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False], 3: [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True], 4: [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False], 5: [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, Tr

In [18]:
#Check total hours of the duty for the week

all_duties_hours = 0
for duty_id in range(len(duty_details)):
    req_skill = duty_details[duty_id][0]
    daily_staff_req = duty_details[duty_id][4]
    hours = duty_details[duty_id][2]
    total_hours = 0
        
    for qty in daily_staff_req:
        total_hours += qty*hours

    all_duties_hours += total_hours

print("total hours for the week",all_duties_hours) 

total hours for the week 857


### Add staff leave details

In [21]:
staffs_on_leave = {1:{3:[0,1,2,3,4,5,6]},
                   9:{1:[3,4]}, 
                   11:{0:[0,1,2],
                      2:[5,6]}} # staff id: week of leave, days on leave


#staff 1 on leave entire week 3
#staff 9 on leave for days 3 and 4 on week 1
#staff 11 on leave for days 0,1,2 on week 0 and days 5,6 on week 2

### Build the Model and add the constraints

In [24]:
# importing google or tools and declaring model template
model = cp_model.CpModel()

# declare empty list that will be used for storing indices for worker-shift-day combination
shiftoptions = {}
# another one to store indices for worker-skill combination


# set number of workers, days and schedules as well as max schedules per day, 
# as well as max shift amount difference per worker
num_weeks = 4
days = 7 
maxhours_per_week = 44
maxhours_difference = 8


# create a tuple as a shift option list index, for each combination of worker, shift and day
# use google or tools to create a boolean variable indicating if given worker works on that day, in that shift
for week in range(num_weeks):
    for staff_id in range(num_staffs):
        for day in range(days):
            for duty_id in range(num_duties):
                shiftoptions[(staff_id,week,day,duty_id)] = model.NewBoolVar(
                    "staff{} week{} day{} duty{}".format(staff_id,week,day,duty_id))




# Constraint 1 
#only qualified staff can take duty
for staff_id in range(num_staffs):
    for duty_id in range(num_duties):
        for week in range(num_weeks):
            for day in range(days):
                if staff_duties_quals[staff_id][duty_id] == False:
                    model.Add(shiftoptions[(staff_id,week,day,duty_id)] == 0)

        
        
# Constraint no.2
# worker only working one shift per day
for staff_id in range(num_staffs):
    for week in range(num_weeks):
        for day in range(days):
            # for each shift that has been assigned to the worker in that day, it'll return 0 or 1 (the domains)
            # sum the total shifts assigned to the worker on that day and make sure this is at most 1 
            model.Add(sum(shiftoptions[(staff_id,week,day,duty_id)] for duty_id in range(num_duties)) <= 1)


# constraint for worker only working 44h per week
# Constraint no.3
for week in range(num_weeks):
    for staff_id in range(num_staffs):        
        hours_worked = 0
        for day in range(days):        
            for duty_id in range(num_duties):
                hours_worked += (shiftoptions[(staff_id,week,day,duty_id)] * duty_details[duty_id][2]) # index 2 is duration

        model.Add(hours_worked <= maxhours_per_week)

    
# instead of 1 worker, shift assigned to same number of workers as specified in duty_details[shift][4][day%7]
# Constraint no.4
for week in range(num_weeks):
    for day in range(days):
        for duty_id in range(num_duties):
            # sum of people working is equals to the number required 
            model.Add(sum(shiftoptions[(staff_id,week,day,duty_id)] for staff_id in range(num_staffs)) == duty_details[duty_id][4][day%7])        
        
     
        
# Constraint no.5, staff on leave cannot work

for staff_id in staffs_on_leave.keys(): #for each staff that are on leave
    for week in staffs_on_leave[staff_id].keys(): #for each week they are on leave 
        for day in staffs_on_leave[staff_id][week]: #for each day they are on leave
            for duty_id in range(num_duties):
                model.Add(shiftoptions[(staff_id,week,day,duty_id)] == 0)

                

    
            
# soft constraint to minimise working hours difference
minhours_perstaff = (all_duties_hours // num_staffs) - (maxhours_difference // 2)
print("min hours per staff:",minhours_perstaff)
maxhours_perstaff = minhours_perstaff + maxhours_difference


for week in range(num_weeks):
    for staff_id in range(num_staffs):
        shiftsassigned = 0
        hours_worked = 0
        leave_hours = 0
        
        if (staff_id in staffs_on_leave.keys()) and (week in staffs_on_leave[staff_id].keys()):
            days_on_leave = len(staffs_on_leave[staff_id][week])
            leave_hours = days_on_leave * 8
        
        if leave_hours > maxhours_perstaff: #cannot be above 44
            # print(staff_id,week)
            leave_hours = maxhours_perstaff
        
        for day in range(days):
            for duty_id in range(num_duties):
                shiftsassigned += shiftoptions[(staff_id,week,day,duty_id)]
                hours_worked += (shiftoptions[(staff_id,week,day,duty_id)] * duty_details[duty_id][2]) # index 2 is duration

#         print(maxhours_perstaff - leave_hours)
        model.Add((minhours_perstaff - leave_hours) <= hours_worked)
        model.Add(hours_worked <= (maxhours_perstaff - leave_hours))


    

min hours per staff: 30


### Run the solver and store the solution

In [23]:
# keep track of which staff work in which shift
schedule_by_week = {}

class ShiftSolutionPrinter (cp_model.CpSolverSolutionCallback):
    
    def __init__(self, shiftoptions, num_staffs, num_weeks, days, num_duties, solution_limit):
        val = cp_model.CpSolverSolutionCallback.__init__(self)
        self._shiftoptions = shiftoptions
        self._num_staffs = num_staffs
        self._days = days
        self._num_weeks = num_weeks
        self._num_duties = num_duties
        self._solution_count = 0
        self._solution_limit = solution_limit
        
    def on_solution_callback(self):
        self._solution_count += 1
        print("\nsolution " + str(self._solution_count))
        for week in range(self._num_weeks):
            schedule_by_day = {}
            
            print("week " + str(week))
            for day in range(self._days):
                daily_staffs_shift = {} # staff:[duties that they work]

                print("day " + str(day))
                for staff_id in range(self._num_staffs):
                    duties = []
                    is_working = False
                    for duty_id in range(self._num_duties):
                        if self.Value(self._shiftoptions[(staff_id,week,day,duty_id)]):
                            is_working = True
                            duties.append(duty_id)
    #                             print("staff id " +str(x) +" works on day " + str(y) +" for duty id " + str(z))

    #                     if not is_working:
    #                         print('  Worker {} does not work'.format(x))

                    print("for day",str(day)," staff ",str(staff_id),"works", str(duties))
                    daily_staffs_shift[staff_id] = duties

                schedule_by_day[day] = daily_staffs_shift
            schedule_by_week[week] = schedule_by_day

        if self._solution_count >= self._solution_limit:
            print('Stop search after %i solutions' % self._solution_limit)
            self.StopSearch()
        
    def solution_count(self):
        return self._solution_count

# solve the model
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0

# solve it and check if solution was feasible
solution_limit = 5 # we want to display 1 feasible results (the first one in the feasible set)
solution_printer  = ShiftSolutionPrinter(shiftoptions, num_staffs, num_weeks,
                                        days, num_duties, solution_limit)

solver.Solve(model,solution_printer)

print('\nStatistics')
print('  - conflicts      : %i' % solver.NumConflicts())
print('  - branches       : %i' % solver.NumBranches())
print('  - wall time      : %f s' % solver.WallTime())
print('  - solutions found: %i' % solution_printer.solution_count())


print(schedule_by_week) #format {week1:{day1:{staff1:shift, staff2:shift}, day:{}  }, week2:...}


solution 1
week 0
day 0
for day 0  staff  0 works []
for day 0  staff  1 works [15]
for day 0  staff  2 works [23]
for day 0  staff  3 works []
for day 0  staff  4 works []
for day 0  staff  5 works []
for day 0  staff  6 works [19]
for day 0  staff  7 works [20]
for day 0  staff  8 works []
for day 0  staff  9 works [27]
for day 0  staff  10 works [9]
for day 0  staff  11 works []
for day 0  staff  12 works [2]
for day 0  staff  13 works []
for day 0  staff  14 works [0]
for day 0  staff  15 works [17]
for day 0  staff  16 works [10]
for day 0  staff  17 works [0]
for day 0  staff  18 works [6]
for day 0  staff  19 works [26]
for day 0  staff  20 works [14]
for day 0  staff  21 works [0]
for day 0  staff  22 works [21]
for day 0  staff  23 works []
for day 0  staff  24 works [4]
day 1
for day 1  staff  0 works [4]
for day 1  staff  1 works []
for day 1  staff  2 works [2]
for day 1  staff  3 works [27]
for day 1  staff  4 works [10]
for day 1  staff  5 works [9]
for day 1  staff  6 w

for day 4  staff  10 works []
for day 4  staff  11 works []
for day 4  staff  12 works [20]
for day 4  staff  13 works []
for day 4  staff  14 works [9]
for day 4  staff  15 works [19]
for day 4  staff  16 works [15]
for day 4  staff  17 works []
for day 4  staff  18 works [10]
for day 4  staff  19 works [21]
for day 4  staff  20 works []
for day 4  staff  21 works []
for day 4  staff  22 works [0]
for day 4  staff  23 works [14]
for day 4  staff  24 works []
day 5
for day 5  staff  0 works [21]
for day 5  staff  1 works [6]
for day 5  staff  2 works [5]
for day 5  staff  3 works []
for day 5  staff  4 works [1]
for day 5  staff  5 works []
for day 5  staff  6 works [3]
for day 5  staff  7 works [13]
for day 5  staff  8 works [16]
for day 5  staff  9 works []
for day 5  staff  10 works []
for day 5  staff  11 works []
for day 5  staff  12 works []
for day 5  staff  13 works [18]
for day 5  staff  14 works []
for day 5  staff  15 works []
for day 5  staff  16 works []
for day 5  staff  

### Validating solver met the constraints

In [25]:
#check schedule validity
#reformat schedule to be a dictionary where each key: value is as follows
# staff_id : [ [week1 schedules], [week2 schedules]...]

schedule_by_staff = {} 

for week in range(num_weeks):
    schedule_by_day = schedule_by_week[week]
    for day in range(len(schedule_by_day)):

        daily_schedule = schedule_by_day[day]

        for staff in range(len(daily_schedule)):
            duty_for_that_staff = daily_schedule[staff]

            if staff in schedule_by_staff.keys():
                schedule_by_staff[staff].append(duty_for_that_staff)

            else:
                schedule_by_staff[staff] = [duty_for_that_staff]
    

print(schedule_by_staff)
#staff 0, works on nothing on monday, duty no.20 tues etc., all the way till day 28

{0: [[], [0], [17], [0], [], [], [9], [6], [26], [], [21], [9], [], [], [0], [21], [20], [0], [], [], [8], [4], [14], [], [14], [0], [13], []], 1: [[], [2], [9], [], [10], [9], [], [26], [14], [], [], [2], [], [22], [], [20], [6], [10], [26], [13], [], [], [], [], [], [], [], []], 2: [[2], [0], [], [0], [14], [5], [], [4], [10], [17], [6], [], [3], [], [15], [6], [0], [], [0], [5], [], [20], [4], [19], [0], [], [], [7]], 3: [[], [], [26], [27], [27], [26], [], [9], [], [27], [19], [], [5], [], [], [27], [27], [27], [27], [], [], [2], [27], [4], [], [27], [], []], 4: [[0], [], [21], [0], [26], [], [8], [], [15], [19], [26], [10], [], [], [6], [], [15], [0], [10], [1], [], [6], [21], [0], [15], [2], [], []], 5: [[0], [4], [6], [], [], [24], [8], [20], [], [0], [], [23], [], [26], [26], [], [0], [2], [0], [], [], [10], [0], [2], [], [17], [1], []], 6: [[], [9], [14], [26], [20], [], [], [2], [2], [], [], [], [6], [9], [23], [10], [], [21], [0], [16], [], [23], [26], [15], [4], [], [24], [

In [29]:
# check constraint 1 is met
# all duties assigned only to qualified staffs

for staff_id in range(num_staffs):
    
    assigned_duties = schedule_by_staff[staff_id]
    qualified_duties = staff_qualifications[staff_id]

    for day in range(days):
        #duty is a list with 1 or more element
        assigned_duty = assigned_duties[day]

        if len(assigned_duty) > 0:
            duty_id = assigned_duty[0] 
            req_skill = duty_details[duty_id][0]

            if req_skill not in qualified_duties:
                print("on day",day,", staff",staff_id,"is unqualified for duty", duty_id,", req skill =",req_skill)
                print("staff_quals:",qualified_duties)
                print("==================")

In [30]:
# check constraint 2 that staff only works 1 shift per day
for staff_id in range(num_staffs):
    for duty in schedule_by_staff:
        assigned_duties = schedule_by_staff[staff_id]
        
        for duty in assigned_duties:
            
            if len(duty) > 1:
                print("staff",staff_id,"works multiple shifts")
                
#if print nothing it's ok                

In [31]:
# check constraint 3
# 44h per week

all_total_hours = 0 
for staff_id in range(num_staffs):
    assigned_duties = schedule_by_staff[staff_id]
    week_num = -1
    weekly_hours = 0
    
    for day in range(days*num_weeks):
        daily_duty = assigned_duties[day]
        
        if day%7 == 0: #new week
            week_num += 1
            weekly_hours = 0
            
        if len(daily_duty)>0: 
            duty_id = daily_duty[0] #means working on that day
            hours = duty_details[duty_id][2]
            weekly_hours += hours
            
        if day%7 == 6: #sunday
            if weekly_hours > 44:
                print("=========================================== This Staff Works Longer than 44h ========================")
            
            print("staff {} works {} hours on week {}".format(staff_id,weekly_hours, week_num))
            all_total_hours += weekly_hours
    
print("total hours of all duties:",all_total_hours) 

staff 0 works 34 hours on week 0
staff 0 works 34 hours on week 1
staff 0 works 35 hours on week 2
staff 0 works 37 hours on week 3
staff 1 works 36 hours on week 0
staff 1 works 32 hours on week 1
staff 1 works 37 hours on week 2
staff 1 works 0 hours on week 3
staff 2 works 37 hours on week 0
staff 2 works 36 hours on week 1
staff 2 works 36 hours on week 2
staff 2 works 37 hours on week 3
staff 3 works 38 hours on week 0
staff 3 works 32 hours on week 1
staff 3 works 36 hours on week 2
staff 3 works 34 hours on week 3
staff 4 works 38 hours on week 0
staff 4 works 34 hours on week 1
staff 4 works 36 hours on week 2
staff 4 works 38 hours on week 3
staff 5 works 33 hours on week 0
staff 5 works 32 hours on week 1
staff 5 works 34 hours on week 2
staff 5 works 37 hours on week 3
staff 6 works 35 hours on week 0
staff 6 works 33 hours on week 1
staff 6 works 35 hours on week 2
staff 6 works 38 hours on week 3
staff 7 works 37 hours on week 0
staff 7 works 34 hours on week 1
staff 7 wor

In [32]:
#reformat the schedule to be by duty
schedule_by_duty = {}

for week in range(num_weeks):
    schedule_by_day = schedule_by_week[week]

    for day in range(days):
        daily_schedule = schedule_by_day[day]
    #     print("day",day,":",daily_schedule) #staff 0:[shifts], #staff 1:[shifts]

        day_num = week*7 + day
        
        for staff in range(num_staffs):
            duty_list = daily_schedule[staff]
#             print("staff id",staff,"on day",day,duty_list)
            if len(duty_list)>0: #means that staff is working on that day
                duty_id = duty_list[0]

                if duty_id in schedule_by_duty.keys():
                    staffs_working_per_day = schedule_by_duty[duty_id]

                    if day_num in staffs_working_per_day.keys():
                        schedule_by_duty[duty_id][day_num].append(staff)

                    else:
                        schedule_by_duty[duty_id][day_num] = [staff] #store the staff id as list

                else:
                    schedule_by_duty[duty_id] = {day_num:[staff]}

print(schedule_by_duty)
#for duty id 0: {day 0: staff ids that work}

{2: {0: [2], 1: [1], 2: [16], 3: [15], 4: [16], 7: [6], 8: [6], 9: [8], 10: [22], 11: [1], 14: [22], 15: [17], 16: [19], 17: [5], 18: [19], 21: [3], 22: [13], 23: [5], 24: [19], 25: [4]}, 0: {0: [4, 5, 24], 1: [0, 2, 7], 2: [10, 17, 24], 3: [0, 2, 4], 4: [10, 15, 21], 7: [12, 14, 22], 8: [8, 10, 16], 9: [5, 12, 18], 10: [14, 18, 19], 11: [13, 16, 19], 14: [0, 20, 21], 15: [12, 18, 19], 16: [2, 5, 15], 17: [0, 4, 7], 18: [2, 5, 6], 21: [12, 14, 15], 22: [5, 8, 18], 23: [4, 10, 20], 24: [2, 12, 18], 25: [0, 22, 24]}, 14: {0: [7], 1: [15], 2: [6], 3: [14], 4: [2], 7: [15], 8: [1], 9: [17], 10: [8], 11: [7], 14: [8], 15: [13], 16: [22], 17: [16], 18: [13], 21: [24], 22: [0], 23: [24], 24: [0], 25: [20]}, 15: {0: [8], 1: [17], 2: [22], 3: [19], 4: [7], 7: [7], 8: [4], 9: [9], 10: [21], 11: [11], 14: [2], 15: [16], 16: [4], 17: [18], 18: [18], 21: [7], 22: [19], 23: [6], 24: [4], 25: [18]}, 19: {0: [10], 1: [10], 2: [9], 3: [11], 4: [20], 7: [18], 8: [13], 9: [4], 10: [3], 11: [14], 14: [18]

In [34]:
# check constraint 4
# num of staffs per day meet those in duty_details

for duty_id in range(len(duty_details)):
    daily_staff_req = duty_details[duty_id][4] #daily_staff_req is a list
#     print(daily_staff_req)
    
    to_check = False
    
    for req in daily_staff_req:
        if req > 0:
            to_check = True
            
        if to_check:

            for day in range(days):
                req_staffs = daily_staff_req[day]
                staffs_assigned = []
                day_ids = []
                
                for week in range(num_weeks):
                    day_ids.append(week*7 + day) #according to num of weeks
                    
                for d in day_ids:
                    if d in schedule_by_duty[duty_id].keys():
                        staffs_assigned = schedule_by_duty[duty_id][d]
#                         print("for duty",duty_id,", day",d,", staffs assigned",staffs_assigned,", req staffs =",req_staffs)

                        if len(staffs_assigned) != req_staffs:
                            print("TOO MANY OR FEW STAFF")
                            
                    elif req_staffs > 0:
                        print("NO STAFF FOR DUTY {} on day {}!!!".format(duty_id,d))
                        print("required staffs =", req_staffs)


In [35]:
# check constraint 5
# staffs on leave cannot work

for staff_id in staffs_on_leave.keys():
    
    for week in staffs_on_leave[staff_id].keys():
        
        no_work_days = staffs_on_leave[staff_id][week]
        staff_schedule = schedule_by_staff[staff_id]
        print("=====================================================")
        print("staff {} on leave for week {} days {}".format(staff_id,week,no_work_days))
        print(staff_schedule)

        for day in no_work_days:
            day_id = week*7 + day
            if len(staff_schedule[day_id]) > 0:
                print("staff {} is working even though on leave for day {}".format(staff_id, day_id))


staff 1 on leave for week 3 days [0, 1, 2, 3, 4, 5, 6]
[[], [2], [9], [], [10], [9], [], [26], [14], [], [], [2], [], [22], [], [20], [6], [10], [26], [13], [], [], [], [], [], [], [], []]
staff 9 on leave for week 1 days [3, 4]
[[], [27], [19], [], [27], [6], [], [], [27], [15], [], [], [24], [], [27], [27], [4], [27], [], [], [], [9], [], [27], [27], [], [28], []]
staff 11 on leave for week 0 days [0, 1, 2]
[[], [], [], [19], [], [18], [], [27], [27], [6], [], [15], [13], [], [], [19], [27], [], [], [], [], [17], [20], [27], [26], [], [], []]
staff 11 on leave for week 2 days [5, 6]
[[], [], [], [19], [], [18], [], [27], [27], [6], [], [15], [13], [], [], [19], [27], [], [], [], [], [17], [20], [27], [26], [], [], []]
